In [1]:
from math import sqrt, log, exp
from scipy.stats import norm
import numpy as np
import pandas as pd
class BasketOptionPricer:
    def __init__(self, S1,S2, K, T, sigma1,sigma2, r, rho, option_type):
        assert option_type in ('call', 'put'), "Option type must be 'call' or 'put'"
        self.S1 = S1
        self.S2 = S2
        self.K = K
        self.T = T
        self.sigma1=sigma1
        self.sigma2=sigma2
        self.r = r
        self.rho = rho  # Correlation matrix or scalar
        self.option_type = option_type


    def geometric_basket(self):
        n = 2  # Number of assets in basket
        B = (self.S1 * self.S2) ** (1 / n)  # Geometric Mean of Assets

        # Creating the covariance matrix
        sigma = [self.sigma1, self.sigma2]

        # Calculating basket volatility using the covariance matrix
        sigma_b = sqrt(self.sigma1**2 + self.sigma2**2 + 2 * self.rho * self.sigma1 * self.sigma2)/n

        # Adjusted drift term
        mu_b = self.r - 0.5 * sum(sigma[i] ** 2 for i in range(n)) / n + 0.5 * sigma_b ** 2

        # Black-Scholes d1 and d2 parameters
        d1 = (log(B / self.K) + (mu_b + 0.5 * sigma_b ** 2) * self.T) / (sigma_b * sqrt(self.T))
        d2 = d1 - sigma_b * sqrt(self.T)

        # Calculating option price based on option type
        if self.option_type == 'call':
            price = exp(-self.r * self.T) * (B * exp(mu_b * self.T) * norm.cdf(d1) - self.K * norm.cdf(d2))
        else:
            price = exp(-self.r * self.T) * (self.K * norm.cdf(-d2) - B * exp(mu_b * self.T) * norm.cdf(-d1))

        return price

    def price_path(self,M):
        np.random.seed(100)#Fixing the seed for reproducibility
        
        #Drift parameter of Brownian motion
        drift1 = (self.r- 0.5 *self.sigma1**2)*self.T 
        drift2 = (self.r- 0.5 *self.sigma2**2)*self.T 

        #Volatility parameter of Brownian motion
        vol1 = self.sigma1 * np.sqrt(self.T) 
        vol2 = self.sigma2 * np.sqrt(self.T) 

        #generate standard normal random variables with correlarion rho
        Z1 = np.random.randn(M,1)
        Z2 = Z1*self.rho+np.sqrt(1-self.rho**2)*np.random.randn(M,1)


        #generate price path
        S1Path = self.S1* np.exp(drift1 + (vol1*Z1)) 
        S2Path = self.S2* np.exp(drift2 + (vol2*Z2)) 
        return S1Path,S2Path
    

    def arithmetic_basket(self,M,control_variate=False):
        """
        Prices an arithmetic basket option using Monte Carlo simulation,
        with optional control variate technique.
        """
        S1Path, S2Path = self.price_path(M)
        
        # Calculate arithmetic basket option payoff
        if self.option_type == 'call':
            arithPayoff = np.exp(-self.r * self.T) * np.maximum((S1Path + S2Path) / 2  - self.K, 0)
            geoPayoff=np.exp(-self.r * self.T)* np.maximum(np.sqrt(S1Path * S2Path)- self.K,0)
        else:
            arithPayoff = np.exp(-self.r * self.T) * np.maximum(self.K -(S1Path + S2Path) / 2 , 0)
            geoPayoff=np.exp(-self.r * self.T)* np.maximum(self.K-np.sqrt(S1Path * S2Path),0)

        # Standard Monte Carlo without control variate
        if not control_variate:
            Pmean = np.mean(arithPayoff)
            Pstd = np.std(arithPayoff)
            confmc = [Pmean - 1.96 * Pstd / np.sqrt(M), Pmean + 1.96 * Pstd / np.sqrt(M)]
            return Pmean, confmc

        # Control variate using geometric basket price
        else:
            geoPrice = self.geometric_basket()  # Closed form solution for geometric basket
            # Covariance and variance for the control variate technique
            covXY = np.mean(np.multiply(arithPayoff,geoPayoff))- np.mean(arithPayoff)*np.mean(geoPayoff) 
            theta = covXY / np.var(geoPayoff)
            Z = arithPayoff + theta * (geoPrice - geoPayoff)
            Zmean = np.mean(Z)
            Zstd = np.std(Z)
            confcv = [Zmean - 1.96 * Zstd / np.sqrt(M), Zmean + 1.96 * Zstd / np.sqrt(M)]
            return Zmean, confcv




def test_basket_option_pricer():
    parameters = [
        {'S1': 100, 'S2': 100, 'K': 100, 'sigma1': 0.3, 'sigma2': 0.3, 'rho': 0.5, 'type': 'put'},
        {'S1': 100, 'S2': 100, 'K': 100, 'sigma1': 0.3, 'sigma2': 0.3, 'rho': 0.9, 'type': 'put'},
        {'S1': 100, 'S2': 100, 'K': 100, 'sigma1': 0.1, 'sigma2': 0.3, 'rho': 0.5, 'type': 'put'},
        {'S1': 100, 'S2': 100, 'K': 80,  'sigma1': 0.3, 'sigma2': 0.3, 'rho': 0.5, 'type': 'put'},
        {'S1': 100, 'S2': 100, 'K': 120, 'sigma1': 0.3, 'sigma2': 0.3, 'rho': 0.5, 'type': 'put'},
        {'S1': 100, 'S2': 100, 'K': 100, 'sigma1': 0.5, 'sigma2': 0.5, 'rho': 0.5, 'type': 'put'},

        {'S1': 100, 'S2': 100, 'K': 100, 'sigma1': 0.3, 'sigma2': 0.3, 'rho': 0.5, 'type': 'call'},
        {'S1': 100, 'S2': 100, 'K': 100, 'sigma1': 0.3, 'sigma2': 0.3, 'rho': 0.9, 'type': 'call'},
        {'S1': 100, 'S2': 100, 'K': 100, 'sigma1': 0.1, 'sigma2': 0.3, 'rho': 0.5, 'type': 'call'},
        {'S1': 100, 'S2': 100, 'K': 80,  'sigma1': 0.3, 'sigma2': 0.3, 'rho': 0.5, 'type': 'call'},
        {'S1': 100, 'S2': 100, 'K': 120, 'sigma1': 0.3, 'sigma2': 0.3, 'rho': 0.5, 'type': 'call'},
        {'S1': 100, 'S2': 100, 'K': 100, 'sigma1': 0.5, 'sigma2': 0.5, 'rho': 0.5, 'type': 'call'}
    ]

    # Results list
    results = []
    # Test each set of parameters
    for params in parameters:
        pricer = BasketOptionPricer(
            S1=params['S1'],
            S2=params['S2'],
            K=params['K'],
            T=3,
            sigma1=params['sigma1'],
            sigma2=params['sigma2'],  
            r=0.05,
            rho=params['rho'],
            option_type=params['type'],
        )
        M = 100000

        # Compute arithmetic basket option price without control variant
        price_without_cv, conf_without_cv = pricer.arithmetic_basket(M, control_variate=False)
        
        # Compute arithmetic basket option price with control variant
        price_with_cv, conf_with_cv = pricer.arithmetic_basket(M, control_variate=True)
        
        # Compute geometric basket option price
        geometric_price = pricer.geometric_basket()
        
        # Add results to the list
        results.append({
            'S1': params['S1'],
            'S2': params['S2'],
            'K':  params['K'],
            'T':  3,
            'sigma1': params['sigma1'],
            'sigma2': params['sigma2'],
            'r': 0.05,
            'rho': params['rho'],
            'option_type': params['type'],
            'M': 100000,
            'Geometric': geometric_price,
            'Arithmetic MC': price_without_cv,
            'MC Confidence Interval': conf_without_cv,
            'Arithmetic Control Variate': price_with_cv,
            'Control Variate Confidence Interval': conf_with_cv
        })
    
    # Create a DataFrame from the results list
    results_df = pd.DataFrame(results)
    
    # Display the DataFrame
    display(results_df)
    
    #Output the DataFrame to a CSV file
    results_df.to_csv('option_pricing_results.csv', index=False)
    
    return results_df

# 使用Pandas DataFrame显示测试结果
test_basket_option_pricer_df = test_basket_option_pricer()


,S1,S2,K,T,sigma1,sigma2,r,rho,option_type,M,Geometric,Arithmetic MC,MC Confidence Interval,Arithmetic Control Variate,Control Variate Confidence Interval
0,100,100,100,3,0.3,0.3,0.05,0.5,put,100000,11.491573,10.466379,"[10.372464163597392, 10.560293269862136]",10.564394,"[10.552227382333314, 10.576560088823701]"
1,100,100,100,3,0.3,0.3,0.05,0.9,put,100000,12.622350,12.321294,"[12.21651858595268, 12.426069649679278]",12.427533,"[12.424819419885578, 12.430247236005476]"
2,100,100,100,3,0.1,0.3,0.05,0.5,put,100000,6.586381,5.462388,"[5.405552770509975, 5.519223881719145]",5.510405,"[5.50185610188287, 5.51895394038201]"
3,100,100,80,3,0.3,0.3,0.05,0.5,put,100000,4.711577,4.196554,"[4.141729923438758, 4.251377111233433]",4.247496,"[4.239785029583501, 4.255206562715504]"
4,100,100,120,3,0.3,0.3,0.05,0.5,put,100000,21.289105,19.753824,"[19.62071664330017, 19.886931974076283]",19.861795,"[19.845428461443152, 19.878161657741995]"
5,100,100,100,3,0.5,0.5,0.05,0.5,put,100000,23.469148,20.934344,"[20.78864598731562, 21.080042382416053]",21.045977,"[21.017715592848766, 21.074237619881607]"
6,100,100,100,3,0.3,0.3,0.05,0.5,call,100000,22.102093,24.464985,"[24.22366085195602, 24.706309900755592]",24.517974,"[24.486747651518087, 24.549201160535496]"
7,100,100,100,3,0.3,0.3,0.05,0.9,call,100000,25.878826,26.286070,"[26.011099992841423, 26.561040197293558]",26.361981,"[26.355553179372617, 26.368408286710274]"
8,100,100,100,3,0.1,0.3,0.05,0.5,call,100000,17.924737,19.463995,"[19.29075778009966, 19.63723214203494]",19.456684,"[19.437467787993604, 19.475900723786495]"
9,100,100,80,3,0.3,0.3,0.05,0.5,call,100000,32.536256,35.409320,"[35.14041746140057, 35.67822194952601]",35.411494,"[35.37925708553135, 35.443731307561265]"
